In [ ]:
import tensorflow as tf

# Get the GPU device name. 
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import os
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


Found GPU at: /device:GPU:0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Section

In [ ]:
embed_pick=pd.read_pickle("/content/drive/My Drive/title_embeddings.pkl") #load the embeddings of the json page title
brand_dict=pd.read_pickle("/content/drive/My Drive/brand_dict.pkl") #load the embeddings of the json page title
model_dict=pd.read_pickle("/content/drive/My Drive/model_dict.pkl") #load the embeddings of the json page title

In [ ]:
import pandas as pd
non_match_pairs=set()
non_match_pairs_by_brand=dict()
match_pairs=set()
t_data=pd.read_csv("/content/drive/My Drive/ground_truth.csv")
for index, row in t_data.iterrows():
    if row['label']==1 and (brand_dict[row['left_spec_id']]!=brand_dict[row['right_spec_id']]):
        print("Mismatch (which we fix)")
        print(row['left_spec_id'])
        print(row['right_spec_id'])
        t_data.loc[index,'label']=0#We fix it as we think it is an error from the ground truth...!
    if row['label']==1:
        match_pairs.add(row['left_spec_id']+"SEPARATOR"+row['right_spec_id'])
    elif row['label']==0 and (brand_dict[row['left_spec_id']]==brand_dict[row['right_spec_id']]):
        non_match_pairs.add(row['left_spec_id']+"SEPARATOR"+row['right_spec_id'])
        if not brand_dict[row['left_spec_id']] in non_match_pairs_by_brand:
            non_match_pairs_by_brand[brand_dict[row['left_spec_id']]]=1
        else:
            non_match_pairs_by_brand[brand_dict[row['left_spec_id']]]+=1
match_pairs=list(match_pairs)
non_match_pairs=list(non_match_pairs)
print(len(match_pairs))
print(len(non_match_pairs))
print(non_match_pairs_by_brand)

Mismatch (which we fix)
www.ebay.com/21318
www.ohc24.ch/797
1073
9010
{'elo': 329, 'dell': 3013, 'nec': 340, 'philips': 1478, 'lg': 8, 'samsung': 381, 'apple': 143, 'rockwell': 374, 'AOC': 198, 'hp': 205, 'benq': 56, 'asus': 2239, 'lenovo': 6, '3M': 228, 'acer': 12}


In [ ]:
train_data=pd.DataFrame(columns=['Left_Title', 'Right_Title', 'Label'])#create a new dataframe

In [ ]:
import random
from difflib import SequenceMatcher
random.shuffle(non_match_pairs)
random.shuffle(match_pairs)
for_non_train=list()
unexpected_count=0
##repace the left and right page titles with their bert embeddings
for row in match_pairs:
    left=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[0].replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[1].replace("/","//")].embeddings
    train_data=train_data.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 1 }, ignore_index=True)
    if model_dict[row.split("SEPARATOR")[0]]!=model_dict[row.split("SEPARATOR")[1]] and SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio() < 0.5:
        print(model_dict[row.split("SEPARATOR")[0]]+"----"+model_dict[row.split("SEPARATOR")[1]]+"-----"+str(SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio()))
        unexpected_count+=1
for row in non_match_pairs:
    left=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[0].replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[1].replace("/","//")].embeddings
    if model_dict[row.split("SEPARATOR")[0]]==model_dict[row.split("SEPARATOR")[1]] or SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio() > 0.5:
        train_data=train_data.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 0 }, ignore_index=True)
    #for_non_train.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 0 })
train_data = train_data.sample(frac=1).reset_index(drop=True)
print(str(unexpected_count))


unknown----e171m-----0.0
u2410----464-7346-----0.15384615384615385
17----m7649-----0.2857142857142857
17----1703fps-----0.4444444444444444
e171m----nec-60003581-nec-----0.19047619047619047
24----242g5djeb-----0.36363636363636365
27----273v5lhsb-----0.36363636363636365
17----e176fpc-----0.4444444444444444
17----e176fpc-----0.4444444444444444
m170----3m-----0.3333333333333333
60002937----ex231w-----0.2857142857142857
2711----display-----0.0
a1082----m9178ll/a-----0.14285714285714285
17----e176fpc-----0.4444444444444444
24----242g5djeb-----0.36363636363636365
p1913----u2312h-----0.18181818181818182
p1913----469-3133-----0.46153846153846156
vs239h-p----23-----0.4
3m----m150-----0.3333333333333333
17----m7649-----0.2857142857142857
1703fps----17-----0.4444444444444444
u2410----464-7346-----0.15384615384615385
a1082----m9178ll/a-----0.14285714285714285
m1700ss----3m-----0.2222222222222222
24----e2483hs-b1-----0.3333333333333333
1715l----e603162-----0.16666666666666666
cd/mâ²,----e1913s-----0

In [ ]:
train_data

,Left_Title,Right_Title,Label
0,"[0.088613994, 0.14435871, 0.5650382, 0.3282786...","[-0.42882594, 0.9942055, -0.4900484, -0.412292...",0
1,"[-0.058767647, 0.91274136, 0.96029824, -0.1463...","[-0.08342198, 0.92886037, 0.96044433, -0.15415...",1
2,"[-0.11800972, 0.55572134, 0.24172956, 0.007664...","[0.27681893, 0.6645248, -0.13211994, -0.070709...",0
3,"[-0.28402418, 0.89129025, 0.7731053, -0.085780...","[-0.17587374, 0.5667431, 0.3828637, -0.1460588...",1
4,"[0.041627787, 1.0514015, 0.7180133, -0.1084840...","[-0.20509768, 1.1495423, 0.309912, -0.12183996...",0
...,...,...,...
3365,"[0.11968633, 1.4369253, 0.12666036, 0.05394667...","[0.14690232, 1.4427694, -0.096945256, -0.29211...",1
3366,"[-0.0039561195, 1.5146664, -0.19067612, -0.100...","[0.08381113, 1.5611442, -0.40741652, -0.177325...",1
3367,"[0.0905031, 0.8045323, -0.41508594, -0.0651109...","[-0.119930334, 0.73516226, -0.52910495, -0.427...",1
3368,"[0.153148, 0.36515096, 0.1425597, 0.28826874, ...","[0.12935777, 1.1141309, -0.017137086, 0.316749...",0


In [ ]:
##find the difference between the embeddings
train_data['difference'] = [abs(x - y) for x,y in zip(train_data['Left_Title'], train_data['Right_Title'])]


Resultant vector : Difference between left spec and right spec data



In [ ]:
train_data.head()


,Left_Title,Right_Title,Label,difference
0,"[0.088613994, 0.14435871, 0.5650382, 0.3282786...","[-0.42882594, 0.9942055, -0.4900484, -0.412292...",0,"[0.51743996, 0.8498468, 1.0550866, 0.74057114,..."
1,"[-0.058767647, 0.91274136, 0.96029824, -0.1463...","[-0.08342198, 0.92886037, 0.96044433, -0.15415...",1,"[0.024654336, 0.016119003, 0.00014609098, 0.00..."
2,"[-0.11800972, 0.55572134, 0.24172956, 0.007664...","[0.27681893, 0.6645248, -0.13211994, -0.070709...",0,"[0.39482865, 0.10880345, 0.3738495, 0.07837486..."
3,"[-0.28402418, 0.89129025, 0.7731053, -0.085780...","[-0.17587374, 0.5667431, 0.3828637, -0.1460588...",1,"[0.10815044, 0.32454717, 0.39024162, 0.0602788..."
4,"[0.041627787, 1.0514015, 0.7180133, -0.1084840...","[-0.20509768, 1.1495423, 0.309912, -0.12183996...",0,"[0.24672547, 0.098140836, 0.4081013, 0.0133558..."


# Simple KNN

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier



In [ ]:
labels= train_data["Label"].to_list()
data_df = train_data["difference"].to_list()

In [ ]:
len(data_df[0])

768

In [ ]:
##split the data set 
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.33, random_state=1000)
for item in for_non_train:
    Emb_test.append(list(abs(x - y) for x,y in zip(item['Left_Title'], item['Right_Title'])))
    Emb_test_label.append(0)

In [ ]:
# k = 3
#train
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(Emb_train, Emb_train_label)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Save File

In [ ]:
import pickle 

In [ ]:


# Its important to use binary mode 
knnPickle = open('/content/drive/My Drive//knn', 'wb') 

# source, destination 
pickle.dump(classifier, knnPickle)                      




In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(max_iter=100).fit(Emb_train, Emb_train_label)

# Its important to use binary mode 
hgbPickle = open('/content/drive/My Drive//hgb', 'wb') 

# source, destination 
pickle.dump(hgb, hgbPickle)   

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('/content/drive/My Drive/knn', 'rb'))

In [ ]:
#print(Emb_test[0])
#print(Emb_test[2000])

In [ ]:
pred=loaded_model.predict(Emb_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print('Accuracy is: ',accuracy_score(Emb_test_label, pred))
con_mat = confusion_matrix(Emb_test_label, pred, [0, 1])

print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

Accuracy is:  0.7439353099730458
              precision    recall  f1-score   support

     class_0       0.80      0.84      0.82       768
     class_1       0.60      0.53      0.56       345

    accuracy                           0.74      1113
   macro avg       0.70      0.69      0.69      1113
weighted avg       0.74      0.74      0.74      1113



In [ ]:
pred=hgb.predict(Emb_test)
print('Accuracy is: ',accuracy_score(Emb_test_label, pred))
con_mat = confusion_matrix(Emb_test_label, pred, [0, 1])

print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

Accuracy is:  0.7538185085354897
              precision    recall  f1-score   support

     class_0       0.76      0.93      0.84       768
     class_1       0.70      0.36      0.48       345

    accuracy                           0.75      1113
   macro avg       0.73      0.65      0.66      1113
weighted avg       0.74      0.75      0.73      1113



In [ ]:
inference_data=pd.read_csv("/content/drive/My Drive/pairs_to_evaluate.csv")

In [ ]:
output=pd.DataFrame(columns=['left_spec_id', 'right_spec_id'])#create a new dataframe
print("left_spec_id","right_spec_id")
for index, row in inference_data.iterrows():
    a=row["left"]
    b=row["right"]
    left=embed_pick.loc[embed_pick['spec'] == a.replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == b.replace("/","//")].embeddings
    difference=[abs(x - y) for x,y in zip(left, right)]
    pred=hgb.predict(difference)
    if pred==1:
        output.append({'left_spec_id': a.replace("/","//"), 'right_spec_id': b.replace("/","//")}, ignore_index=True)
        #print(a.replace("/","//")+","+b.replace("/","//"))
    if index%100==0:
        print(index)
output.to_csv("/content/output.csv", sep=',',encoding='utf-8', index=False)


Streaming output truncated to the last 5000 lines.
829800
829900
830000
830100
830200
830300
830400
830500
830600
830700
830800
830900
831000
831100
831200
831300
831400
831500
831600
831700
831800
831900
832000
832100
832200
832300
832400
832500
832600
832700
832800
832900
833000
833100
833200
833300
833400
833500
833600
833700
833800
833900
834000
834100
834200
834300
834400
834500
834600
834700
834800
834900
835000
835100
835200
835300
835400
835500
835600
835700
835800
835900
836000
836100
836200
836300
836400
836500
836600
836700
836800
836900
837000
837100
837200
837300
837400
837500
837600
837700
837800
837900
838000
838100
838200
838300
838400
838500
838600
838700
838800
838900
839000
839100
839200
839300
839400
839500
839600
839700
839800
839900
840000
840100
840200
840300
840400
840500
840600
840700
840800
840900
841000
841100
841200
841300
841400
841500
841600
841700
841800
841900
842000
842100
842200
842300
842400
842500
842600
842700
842800
842900
843000
843100
843200
8433